
## Chain of Thought (CoT) Creation

In this notebook, we aim to create a Chain of Thought (CoT) for clinical trial studies using HuggingFace's transformers and other relevant libraries. Below is a step-by-step guide to our workflow:

1. **Setup and Initialization**:
  - We initialize the ChromaDB client and load the SentenceTransformer model for encoding queries.
  - We define a function to fix invalid JSON strings, which is crucial for handling the metadata of clinical trial studies.

2. **Retrieving Relevant Studies**:
  - We define a function `retrieve_relevant_studies` that queries the ChromaDB collection to find studies relevant to a given query, excluding the study already present in the query.

3. **Crafting Context from Studies**:
  - We define a function `craft_context_from_studies_documents` to create a context string from the documents of related studies. This context is used to provide examples in the CoT creation process.

4. **Generating Messages for CoT**:
  - We define a function `get_messages_for_create_CoT` that generates the system and user messages required for creating a CoT. These messages include the study title, description, and desired criteria.

5. **Prompt Creation**:
  - We define a function `get_prompt_from_studies` that uses the above functions to generate the complete prompt for a given study. This prompt includes the context from related studies and the task instructions for generating the CoT.

6. **Model Inference**:
  - We load the HuggingFace model and tokenizer, and define a function `pipe` to generate the CoT using the model. The function takes the generated messages as input and returns the model's output.
  - For Gemini, can use the function in the gemini section to generate the CoT.

By following this workflow, we can systematically generate a Chain of Thought for clinical trial studies, leveraging the capabilities of HuggingFace's transformers and other relevant tools.
```

In [2]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path="./clinical_trials_chroma")
embed_model = SentenceTransformer("malteos/scincl")
collection = client.get_or_create_collection("clinical_trials_studies")

/home/swiss/miniconda3/envs/ML/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# import re
# def fix_invalid_json(input_str):
#     ## add double quotes around elements inside square brackets if not already quoted
#     fixed_str = re.sub(r'(?<=\[)([^\[\],]+)(?=\])', lambda x: '"' + x.group(0).strip() + '"', input_str)
    
#     ## add double quotes around words in Conditions and Interventions
#     fixed_str = re.sub(r'(?<=\[)([^\"\]]+?)(?=\])', lambda x: '"' + x.group(0).strip().replace(", ", '", "') + '"', fixed_str)
    
#     ## fix key-value pairs inside Interventions
#     fixed_str = re.sub(r'"([A-Za-z]+): ([A-Za-z0-9\s]+)"', r'"\1: \2"', fixed_str)
    
#     # fix dictionary keys
#     fixed_str = re.sub(r'(?<!")(\b[A-Za-z_]+\b)(?=\s*:)', r'"\1"', fixed_str)
    
#     return fixed_str

In [4]:
# retrieve relevent studies from chromadb but exclude the ones that are already is the query
def retrieve_relevant_studies(query, existing_study, n_results=5):
    query_embedding = embed_model.encode(query).tolist()
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results + 1,
    )
    
    filtered_results = []
    for id, distance, document in zip(results['ids'][0], results['distances'][0], results['documents'][0]):
        if id != existing_study:
            filtered_results.append({
                "id": id,
                "distance": distance,
                "document": document,
            })
        
        if len(filtered_results) == n_results:
            break
    
    return filtered_results

print(
    retrieve_relevant_studies("Effect of Kinesiotaping on Edema Management, Pain and Function on Patients With Bilateral Total Knee Arthroplasty [SEP] After being informed about the study and potential risk, all patients undergoing inpatient rehabilitation after bilateral total knee arthroplasty will have Kinesio(R)Tape applied to one randomly selected leg while the other leg serves as a control. Measurement of bilateral leg circumference, knee range of motion, numerical rating scale for pain, and selected questions from the Knee Injury and Osteoarthritis Outcome Score will occur at regular intervals throughout the rehabilitation stay. Patients will receive standard rehabilitation.", 
                              "NCT05013879"))

[{'id': 'NCT03484429', 'distance': 108.2900390625, 'document': '{"metadata": {"NCT_ID": "NCT03484429", "Brief_Title": "Postoperative Peripheral Nerve Stimulation for Management of Post-amputation Pain", "Official_title": "Effects of Postoperative Percutaneous Peripheral Nerve Stimulation on Acute and Chronic Amputation Pain", "Conditions": ["Phantom Limb Pain", "Postoperative Pain", "Neuroma", "Acute Pain", "Chronic Pain", "Residual Limbs", "Amputation"], "Interventions": ["Device: Peripheral nerve stimulation", "Other: Standard Medical Therapy"], "Location_Countries": ["United States"], "Study_Design": {"Study_Type": "INTERVENTIONAL", "Phase": ["NA"], "Primary_Purpose": "TREATMENT", "Allocation": "RANDOMIZED", "Interventional Model": "PARALLEL", "Masking": "NONE"}}, "description": "#Study Description \\nBrief Summary \\nLimb loss is frequently associated with postamputation pain that can be challenging to treat and often involves opioids. Advances in the field of neuromodulation has l

In [5]:
import json
import json_repair
def related_studies_template(title: str, description: str, criteria: str):
    return f"""Example Title: {title}
Example Description: {description}
Example Criteria: {criteria}
"""

def craft_context_from_studies_documents(related_studies: list[str]):
    json_related_studies = [json.loads(i) for i in related_studies]
    context = ""
    for i in json_related_studies:
        title = i.get('metadata', {}).get('Official_title', "")
        description = i.get('description', "")
        criteria = i.get('criteria', "")
        if title and description:
            context += f"""<STUDY>
{related_studies_template(title, description, criteria)}
</STUDY>"""
    return context

def user_prompt_template(encoded_related_studies: str, title: str, description: str, desired_criteria: str):
    user_prompt_template = """<EXAMPLE_STUDIES>{encoded_related_studies}</EXAMPLE_STUDIES>

Title: {title}
Description: {description}
Desired criteria: {desired_criteria}

Task Instructions:
1. Derive a step-by-step justification starting from the "Title" and "Description" provided, gradually building up to support the "Desired criteria".
2. Clearly explain the rationale behind each parameter of all criteria, including values, thresholds, and other specific details.
3. Could use example studies (in the <EXAMPLE_STUDIES> section) if they support your justifications, but ensure the reasoning is well-explained and relevant to the study's context.
4. Avoid mentioning that the criteria were already provided, and please do not cite the "Desired criteria" directly in your justification.
5. You should give the justification first before giving out the criteria.

Response Format:
<STEP-BY-STEP-JUSTIFICATION>
Your long step by step detailed logical justification here.
</STEP-BY-STEP-JUSTIFICATION>
"""

    return user_prompt_template.format(encoded_related_studies=encoded_related_studies, title=title, description=description, desired_criteria=desired_criteria)

system_prompt = "You are a justifier chatbot designed to generate step-by-step justifications that derived form the Title and Description of a study and then gradually build up to the Desired criteria. Your task is to analyze the title and description of a study and build logical, step-by-step justifications that connect the study’s key elements to the desired criteria. Reference related example studies if they reinforce your justifications. You must assume the desired criteria are correct (as it was already reviewed by specialists) and develop arguments to support them based on the study context and relevant research insights."

def get_messages_for_CoT_huggingface(encoded_related_studies: str, title: str, description: str, desired_criteria: str):
    return [
        {"role": "system", "content": "You are a justifier chatbot designed to generate step-by-step justifications that derived form the Title and Description of a study and then gradually build up to the Desired criteria. Your task is to analyze the title and description of a study and build logical, step-by-step justifications that connect the study’s key elements to the desired criteria. Reference related example studies if they reinforce your justifications. You must assume the desired criteria are correct (as it was already reviewed by specialists) and develop arguments to support them based on the study context and relevant research insights."},
        {"role": "user", "content": user_prompt_template(encoded_related_studies, title, description, desired_criteria)},
    ]
    

def get_info_for_prompt_gen(study_info: dict):
    metadata = json_repair.loads(study_info.get('metadata'))
    title = metadata.get('Official_title')
    description = study_info.get('data')
    study_id = metadata.get('NCT_ID')
    desired_criteria = study_info.get('criteria')

    # Ensure we have the minimum required information
    if not title or not description or not desired_criteria or not study_id:
        print(f"Skipping study {study_id}: Missing title or description or desired criteria or study id")
        return None

    query = f'{title} [SEP] {description}'
    relevant_studies = retrieve_relevant_studies(query, study_id)
    encoded_related_studies = craft_context_from_studies_documents([i['document'] for i in relevant_studies])
    return encoded_related_studies, title, description, desired_criteria




## Using Llama 3.1

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_id = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16"
number_gpus = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = LLM(model=model_id, tensor_parallel_size=number_gpus, max_model_len=20000)

def pipe(messages):
    sampling_params = SamplingParams(temperature=0, top_p=0.9, max_tokens=4096)
    prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    outputs = llm.generate(prompts, sampling_params)
    return [i.outputs[0].text for i in outputs]

In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Load the JSON data
ravis_dataset = load_dataset("ravistech/clinical-trial-llm-cancer-restructure")

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['encoded_related_studies', 'title', 'description', 'desired_criteria', 'messages'])

for study in tqdm(ravis_dataset['train']):
    print(f"Processing {study}")
    info_for_prompt = get_info_for_prompt_gen(study)
    
    if info_for_prompt:
        encoded_related_studies, title, description, desired_criteria = info_for_prompt
        messages = get_messages_for_CoT_huggingface(encoded_related_studies, title, description, desired_criteria)
        
        # Add the data to the DataFrame
        df = df.append({
            'encoded_related_studies': encoded_related_studies,
            'title': title,
            'description': description,
            'desired_criteria': desired_criteria,
            'messages': messages
        }, ignore_index=True)
        
        print(f"Prompt: {messages}")
        print(f"Response: {pipe(messages)}")

# Save the DataFrame to a CSV file
df.to_csv('responses_gemini.csv', index=False)




## Using Gemini

In [6]:
import time
import pandas as pd
import json
from datasets import load_dataset
from tqdm import tqdm
import vertexai
from vertexai.batch_prediction import BatchPredictionJob
import os
import uuid

PROJECT_ID = "PROJECT-ID"  # update with Google Cloud project ID
BUCKET_NAME = "BUCKET-NAME" # create a bucket in Google Cloud Storage first
OUTPUT_DIR = "./output/"
CSV_OUTPUT_PATH = "./responses_gemini.csv"
vertexai.init(project=PROJECT_ID, location="us-central1")

ravis_dataset = load_dataset("ravistech/clinical-trial-llm-cancer-restructure")

batch_prompts = []
info_data = {}  # key is uuid, value is the info for the prompt
for study in tqdm(ravis_dataset['train']):
    info_for_prompt = get_info_for_prompt_gen(study)
    
    if info_for_prompt:
        unique_id = str(uuid.uuid4())  # gen uuid for each entries
        encoded_related_studies, title, description, desired_criteria = info_for_prompt
        messages = user_prompt_template(encoded_related_studies, title, description, desired_criteria)

        request_format = {
            "id": unique_id,
            "request": {
                "contents": [
                    {
                        "role": "user",
                        "parts": [
                            {"text": messages}
                        ]
                    }
                ],
                "system_instruction": {
                    "parts": [
                        {"text": system_prompt}
                    ]
                }
            }
        }
        batch_prompts.append(request_format)
        
        # store info with unique ID for later matching with the response
        info_data[unique_id] = {
            "encoded_related_studies": encoded_related_studies,
            "title": title,
            "description": description,
            "desired_criteria": desired_criteria,
            "messages": messages,
            "response": None
        }

input_jsonl_path = f'gs://{BUCKET_NAME}/prompt_for_batch_gemini_predict.jsonl'
with open('./prompt_for_batch_gemini_predict.jsonl', 'w') as f:
    for prompt in batch_prompts:
        f.write(json.dumps(prompt) + '\n')

# Upload the JSONL file to GCS
!gsutil cp ./prompt_for_batch_gemini_predict.jsonl {input_jsonl_path}

batch_prediction_job = BatchPredictionJob.submit(
    source_model="gemini-1.5-flash-002",
    input_dataset=input_jsonl_path,
    output_uri_prefix=f'gs://{BUCKET_NAME}/output/',
)

print(f"Job resource name: {batch_prediction_job.resource_name}")
print(f"Model resource name with the job: {batch_prediction_job.model_name}")
print(f"Job state: {batch_prediction_job.state.name}")

# Check the job status
while not batch_prediction_job.has_ended:
    time.sleep(5)
    batch_prediction_job.refresh()

# Process the output if the job is successful
if batch_prediction_job.has_succeeded:
    print("Job succeeded!")
    output_location = batch_prediction_job.output_location + "/predictions.jsonl"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    local_output_path = os.path.join(OUTPUT_DIR, "predictions.jsonl")

    # Download the output file from GCS
    !gsutil cp {output_location} {local_output_path}
    print(f"Output file downloaded to: {local_output_path}")
    
    with open(local_output_path, 'r') as f:
        for line in f:
            response_data = json.loads(line)
            unique_id = response_data.get("id")
            response = response_data.get("response", {})
            
            if unique_id in info_data:
                info_data[unique_id]["response"] = response

    df = pd.DataFrame.from_dict(info_data, orient="index")
    df.to_csv(CSV_OUTPUT_PATH, index=False)
    print(f"Responses saved to CSV at: {CSV_OUTPUT_PATH}")
else:
    print(f"Job failed: {batch_prediction_job.error}")

print(f"Job output location: {batch_prediction_job.output_location}")



  0%|          | 5/31938 [00:00<08:26, 63.04it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Copying file://./prompt_for_batch_gemini_predict.jsonl [Content-Type=application/octet-stream]...
- [1 files][114.4 KiB/114.4 KiB]                                                
Operation completed over 1 objects/114.4 KiB.                                    
BatchPredictionJob created. Resource name: projects/216790650521/locations/us-central1/batchPredictionJobs/6987701337194496000
To use this BatchPredictionJob in another session:
job = batch_prediction.BatchPredictionJob('projects/216790650521/locations/us-central1/batchPredictionJobs/6987701337194496000')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/6987701337194496000?project=216790650521
Job resource name: projects/216790650521/locations/us-central1/batchPredictionJobs/6987701337194496000
Model resource name with the job: publishers/google/models/gemini-1.5-flash-002
Job state: JOB_STATE_PENDING
Job succeeded!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Copying gs://ravistech-bucket-swiss/output/prediction-model-2024-11-05T08:57:15.648971Z/predictions.jsonl...
- [1 files][139.4 KiB/139.4 KiB]                                                
Operation completed over 1 objects/139.4 KiB.                                    
Output file downloaded to: ./output/predictions.jsonl
Responses saved to CSV at: ./responses_gemini.csv
Job output location: gs://ravistech-bucket-swiss/output/prediction-model-2024-11-05T08:57:15.648971Z
